Suggestions
use votes as a weighted averages
KNN clustering for classifications

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from pathlib import Path
import os 
import statsmodels.api as sm
import plotly.express as px
import re
import gzip
from urllib.request import urlopen

### Handling nulls

#### Cleaning Review Data set 

In [62]:
#directing to the right file path
os.chdir("/Users/mac/Desktop/Data/CAPSTONE")
cwd = os.getcwd() 

In [86]:
#Opening the reviews,  resource: https://towardsdatascience.com/load-yelp-reviews-or-other-huge-json-files-with-ease-ad804c2f1537
review_df = []
r_dtypes = {"overall": np.float16, 
            "verified": np.int32, 
            "vote": np.int32,
            "reviewTime": np.int32,
            "reviewerID": np.int32,
            "asin": object,
            "reviewerName": object,
            "reviewText":object , 
            "summary": object,     
            "style": object, 
            "image": object, 
           }
with open("Luxury_Beauty.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=r_dtypes, chunksize=1000)
        
    for chunk in reader:
        reduced_chunk = chunk.drop(columns=['unixReviewTime'],axis=1)
        review_df.append(reduced_chunk)
    
review_df = pd.concat(review_df, ignore_index=True)

In [88]:
#checking review_df dataset
review_df.head(10)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,style,image
0,2.0,3,1,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,NaN,NaN
1,5.0,14,1,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,NaN,NaN
2,5.0,NaN,1,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,{'Size:': ' 3.5 oz.'},NaN
3,5.0,NaN,1,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,{'Size:': ' 3.5 oz.'},NaN
4,5.0,NaN,1,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,{'Size:': ' 3.5 oz.'},NaN
5,5.0,NaN,1,"04 14, 2018",AXX5G4LFF12R6,B00004U9V2,Ralla,Ive used this lotion for many years. I try oth...,Ive used this lotion for many years. I try ...,{'Size:': ' 250 g'},NaN
6,5.0,NaN,1,"04 11, 2018",A7GUKMOJT2NR6,B00004U9V2,Lydia Speight,Works great for dry hands.,Five Stars,{'Size:': ' 3.5 oz.'},NaN
7,5.0,NaN,1,"04 11, 2018",A3FU4L59BHA9FY,B00004U9V2,Allen Semer,The best hand cream ever.,Made in the USA,{'Size:': ' 3.5 oz.'},NaN
8,5.0,NaN,1,"04 7, 2018",A1AMNMIPQMXH9M,B00004U9V2,Vets park,LOVE THIS SCENT!! But Crabtree and Evelyn mak...,Moistens and smells good,{'Size:': ' 3.5 oz.'},NaN
9,5.0,NaN,1,"04 6, 2018",A3DMBDTA8VGWSX,B00004U9V2,Cynthia P. Irving,Its a great moisturizer especially for gardners,Five Stars,{'Size:': ' 3.5 oz.'},NaN


#### Data Dictionary

**Review Data** 
- `overall` Rating given by user out of 5.0 (numeric)
- `verified`: Denotes verified purchases or not (numeric) 
- `vote`: Number of users that have liked the review (numeric)
- `reviewTime`: Recorded time of review (numeric)
- `reviewerID`: Unique reviewer ID (object)
- `asin`: Unique product ID (object)
- `reviewerName`: Name given of reviewer (object) 
- `reviewText`: Body of user review (object) 
- `summary`: Title of user review (object)     
- `style`: Dictionary object containing details on the product reviewed (Dictionary)
- `image`: Associated images in JPEGs of the product uploaded by user(object) 

**Meta Data** 
- `category`: object
- `tech1`: object,
- `description`: object,
- `fit`: object,
- `title`: object,
- `also_buy`: list,
- `tech2`: object, 
- `brand`: object,
- `feature`: object,
- `rank`:object,
- `also_view`: list, 
- `details`: object,
- `Shipping Weight`: object,
- `International Shipping`: object,
- `ASIN`: object, 
- `Item model number`: object,
- `main_cat`: object,
- `similar_item`: object,
- `date`: object,
- `price`: np.float32,
- `asin`: object, 
- `imageURL`: list, 
- `imageURLHighRes`: list ,
          

In [89]:
#checking amount of nulls   
nan_count = review_df.isna().sum()
print(nan_count)

overall              0
vote            470939
verified             0
reviewTime           0
reviewerID           0
asin                 0
reviewerName        31
reviewText         400
summary            183
style           323615
image           567210
dtype: int64


`Image`, `Vote`, `Style` all have high levels of null values. However we want to keep `Vote` column by filling with 0s since this is an indication of other users agreeing with the review written. Before we move on we should explore the style column to see if there is any useful information there since data is stored in a dictionary.

In [90]:
#filling Vote column nulls with 0
review_df["vote"].fillna(0, inplace = True)

In [91]:
review_df['vote']= review_df['vote'].str.replace(",","")

In [92]:
review_df['vote']= review_df['vote'].astype("float32")

In [93]:
review_df

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,style,image
0,2.0,3.0,1,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,NaN,NaN
1,5.0,14.0,1,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,NaN,NaN
2,5.0,NaN,1,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,{'Size:': ' 3.5 oz.'},NaN
3,5.0,NaN,1,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,{'Size:': ' 3.5 oz.'},NaN
4,5.0,NaN,1,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,{'Size:': ' 3.5 oz.'},NaN
...,...,...,...,...,...,...,...,...,...,...,...
574623,5.0,NaN,1,"03 20, 2017",AHYJ78MVF4UQO,B01HIQEOLO,Lori Fox,Great color and I prefer shellac over gel,Five Stars,NaN,NaN
574624,5.0,NaN,1,"10 26, 2016",A1L2RT7KBNK02K,B01HIQEOLO,Elena,Best shellac I have ever used. It doesn't tak...,Best shellac I have ever used,NaN,NaN
574625,5.0,NaN,1,"09 30, 2016",A36MLXQX9WPPW9,B01HIQEOLO,Donna D. Harris,Great polish and beautiful color!!,Great polish!,NaN,NaN
574626,1.0,2.0,1,"12 5, 2016",A23DRCOMC2RIXF,B01HJ2UY0W,Y.Y. Chen,"The perfume is good, but the spray head broke ...",Spray head broke off within a month,"{'Size:': ' 1.7 Fluid Ounce', 'Color:': ' Multi'}",NaN


In [94]:
#converting vote column to vote
print(review_df.dtypes)

overall         float16
vote            float32
verified          int32
reviewTime       object
reviewerID       object
asin             object
reviewerName     object
reviewText       object
summary          object
style            object
image            object
dtype: object


In [95]:
review_df.iloc[22620:22625]

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,style,image
22620,5.0,5.0,1,"11 16, 2015",A14OAB0YBN176K,B0002ZW5UQ,P Nitty,"It's funny, I tried this same product about 15...","If you have thinning hair, this is perfect. I...","{'Size:': ' 0.42 oz.', 'Color:': ' Black'}",NaN
22621,4.0,NaN,1,"11 15, 2015",A1MB9LANC755GE,B0002ZW5UQ,kamran,"Very nice. Thanks,",Four Stars,"{'Size:': ' 0.42 oz.', 'Color:': ' Black'}",NaN
22622,5.0,NaN,1,"11 14, 2015",AHYZUR6W4B3EL,B0002ZW5UQ,Zachary Spencer,I honestly love this stuff! It does wonders. E...,I love it!!!!,"{'Size:': ' 0.97 oz.', 'Color:': ' Dark Brown'}",NaN
22623,4.0,2995.0,1,"11 14, 2015",A2J0S1IC4PU9G8,B0002ZW5UQ,Big Stink,"So, if you're bothering to read reviews about ...","Not bad, Toppik, but let's be clear about a fe...","{'Size:': ' 0.42 oz.', 'Color:': ' Dark Brown'}",NaN
22624,5.0,NaN,1,"11 13, 2015",A2N5487XPK9L8E,B0002ZW5UQ,Yaneida Gutierrez,"Love it, does the work well, and I love that I...","Go on, buy it!","{'Size:': ' 0.97 oz.', 'Color:': ' Dark Brown'}",NaN


In [96]:
review_df.describe()

,overall,vote,verified
count,574628.0,103689.000000,574628.000000
mean,NaN,7.399059,0.878032
std,0.0,27.668232,0.327249
min,1.0,2.000000,0.000000
25%,4.0,2.000000,1.000000
50%,5.0,3.000000,1.000000
75%,5.0,6.000000,1.000000
max,5.0,2995.000000,1.000000


In [97]:
review_df

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,style,image
0,2.0,3.0,1,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,NaN,NaN
1,5.0,14.0,1,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,NaN,NaN
2,5.0,NaN,1,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,{'Size:': ' 3.5 oz.'},NaN
3,5.0,NaN,1,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,{'Size:': ' 3.5 oz.'},NaN
4,5.0,NaN,1,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,{'Size:': ' 3.5 oz.'},NaN
...,...,...,...,...,...,...,...,...,...,...,...
574623,5.0,NaN,1,"03 20, 2017",AHYJ78MVF4UQO,B01HIQEOLO,Lori Fox,Great color and I prefer shellac over gel,Five Stars,NaN,NaN
574624,5.0,NaN,1,"10 26, 2016",A1L2RT7KBNK02K,B01HIQEOLO,Elena,Best shellac I have ever used. It doesn't tak...,Best shellac I have ever used,NaN,NaN
574625,5.0,NaN,1,"09 30, 2016",A36MLXQX9WPPW9,B01HIQEOLO,Donna D. Harris,Great polish and beautiful color!!,Great polish!,NaN,NaN
574626,1.0,2.0,1,"12 5, 2016",A23DRCOMC2RIXF,B01HJ2UY0W,Y.Y. Chen,"The perfume is good, but the spray head broke ...",Spray head broke off within a month,"{'Size:': ' 1.7 Fluid Ounce', 'Color:': ' Multi'}",NaN


In [98]:
#sanity check
review_df["vote"].isna().sum()

470939

In [ ]:
#Splitting out Key Value pairs into separate columns for Style
review_df = pd.concat([review_df,review_df["style"].apply(pd.Series)], axis=1)
review_df

In [ ]:
#Removing columns with null values greater than 80% https://stackoverflow.com/questions/43311555/how-to-drop-column-according-to-nan-percentage-for-dataframe 
review_df = review_df.loc[:, review_df.isnull().mean() < .8]
review_df

In [ ]:
review_df.info()

In [80]:
#Removing unnecessary columns and Image
review_df = review_df.drop(columns=["style","Size:"] ,inplace=True, axis=1)

AttributeError: 'NoneType' object has no attribute 'drop'

In [78]:
nan_count = review_df.isna().sum()
print(nan_count)

AttributeError: 'NoneType' object has no attribute 'isna'

In [ ]:
#replacing na values of reviewer name with reviewer ID 
review_df = review_df['reviewerName'].fillna(review_df['reviewerID'], inplace=True)

In [ ]:
nan_count = review_df.isna().sum()
print(nan_count)

In [ ]:
# % of na values of review_df
pct_reviewText = review_df["reviewText"].isna().sum()/review_df.shape[0]*100
print(f"Null values of reviewText makes up {round(pct_reviewText,2)}% of dataset, this is small so we can remove these lines")

In [ ]:
#removing remaining rows with null values and checking row counts
review_df = review_df.dropna()
review_df.shape

In [ ]:
#sanity check
review_df.isna().sum().sum()

### Row Duplicates

Now that we have dealt with the null values we can move onto checking if we have any duplicates of rows. 

In [ ]:
#checking count of duplicated rows 
review_df.duplicated().sum()

In [ ]:
# % of na values of review_df
pct_duplicates = review_df.duplicated().sum()/review_df.shape[0]*100
print(f"Duplicate rows make up {round(pct_duplicates,2)} % of dataset, which is fairly large")

In [ ]:
#looking at duplicates
review_df[review_df.duplicated()].sample(5)

In [ ]:
pct_origin_row = review_df.duplicated().sum() /review_df.duplicated(keep=False).sum()
print(f"Each duplicated row appears close to twice at {round(pct_origin_row,2)} of the dataset, so there was likely a data recording issue.")

In [ ]:
#removing duplicates
review_df = review_df.drop_duplicates(inplace=True)
review_df.shape

In [ ]:
#creating clean review_df and copying in review_df into it 
clean_review_df = pd.DataFrame()
clean_review_df = review_df.copy()

In [ ]:
clean_review_df.isna().sum()

#### Cleaning Metadata Dataset

In [ ]:
#Opening the Meta data 
metadata_df = []
r_dtypes = {"category": object,
            "tech1": object,
            "description": object,
            "fit": object,
            "title": object,
            "also_buy": list,
            "tech2": object, 
            "brand": object,
            "feature": object,
            "rank":object,
            "also_view": list, 
            "details": object,
            "Shipping Weight": object,
            'International Shipping': object,
            "ASIN": object, 
            "Item model number": object,
            "main_cat": object,
            "similar_item": object,
            "date": object,
            "price": np.float32,
            "asin": object, 
            "imageURL": list, 
            "imageURLHighRes": list ,
           }
with open("meta_Luxury_Beauty.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=r_dtypes, chunksize=1000)
        
    for chunk in reader:
        reduced_chunk = chunk.drop(columns=["tech1"],axis=1)
        metadata_df.append(reduced_chunk)
    
metadata_df = pd.concat(metadata_df, ignore_index=True)

In [ ]:
#Row and column count 
metadata_df.shape

In [ ]:
metadata_df.head(10)

In [ ]:
metadata_df.info()

There are many empty columns in the dataset, and many with just empty lists or the same value repeated throughout: category, tech1, fit, tech2, brand, feature, main_cat, date. 

Equally since we wont be needing the image URLs we can remove these columns as well.

In [ ]:
#removing columns 
columns= ["category", "fit", "tech2", "brand", "feature", "main_cat", "date", "imageURL","imageURLHighRes"]

metadata_df.drop(columns,axis=1, inplace=True)

In [ ]:
metadata_df.head(5)

In [ ]:
#applying NaN values in place of empty lists
metadata_df = metadata_df.where(~metadata_df.applymap(lambda x: x == [] or x is None or x == ''))

In [ ]:
#null value count
null_metadata = metadata_df.isna().sum()
null_metadata["price"]

In [ ]:
#metadata_df.explode("also_buy")

In [ ]:
metadata_df[['ranking','remove']] = metadata_df["rank"].str.split(" ", n=1,expand = True)

In [ ]:
#convert ranking column to integer
metadata_df['ranking'] = metadata_df["ranking"].str.replace(",", "", regex=False)
metadata_df["ranking"].fillna(0, inplace = True)
metadata_df["ranking"] =metadata_df["ranking"].astype("int32")

In [ ]:
#remove unnecessary columns
metadata_df.drop(['remove','rank',"similar_item"],axis=1,inplace=True )

In [ ]:
#removing description from list 
metadata_df['description1'] = metadata_df['description'].str.join(', ')

In [ ]:
#taking a look at details column
d = metadata_df["details"].iloc[0]
d.keys()

Taking a look at the values that are contained in the `details` column, we can pass on these since these don't have much relation to contributing to the individual products. If we didnt already have access to the ASIN then we would keep this value. 

In [ ]:
metadata_df.drop(["details","description"],axis=1, inplace=True)

`price` is an object right now, this needs to convert to numeric value. 

In [ ]:
metadata_df["asin"].unique()

In [ ]:
#removing dollar sign from price and replacing those values with 0 
metadata_df['price'] = metadata_df["price"].str.replace("$", "", regex=False)
metadata_df["price"].fillna(0, inplace = True)
metadata_df

In [ ]:
metadata_df["price"] = metadata_df["price"].str.strip()

In [ ]:
#extracting string from

metadata_df['price2']= metadata_df['price'].str.extract(r'(\d*?\.\d{2})', expand=False)
metadata_df['price2'] = metadata_df['price2'].astype(float)

In [ ]:
metadata_df.info()

In [ ]:
metadata_df["asin"].unique()


In [ ]:
len(metadata_df_also_buy)

In [ ]:
#maybe do something about the missing price values?
null_metadata["price"] 

In [ ]:
#dictionary of value counts of also buy.
my_count = metadata_df_also_buy.value_counts()
dict_alsobuy = dict(my_count)

In [ ]:
#creating new column for also buy counts
metadata_df["also_buy_counts"] = metadata_df["asin"].map(dict_alsobuy)

In [ ]:
metadata_df.info()

In [282]:
metadata_df_unique.shape

(12111,)

In [ ]:
metadata_df

In [ ]:
list1 = metadata_df["price"]
for i in list1:

    try:
        Output = float(i)
    except ValueError:
        Output = float(i.replace("''", ''))
        print(output)
 


In [ ]:
test1 = metadata_df["price"].iloc[:1]
print(float(test1[0]))

#To-do </br>
rank - only keep number done </br> 
description, - take out of list done </br>
details - exploring done </br>
price - removing the $ and , and converting to float 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12299 entries, 0 to 12298
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         12299 non-null  object
 1   also_buy      7724 non-null   object
 2   also_view     9046 non-null   object
 3   price         12299 non-null  object
 4   asin          12299 non-null  object
 5   ranking       12201 non-null  object
 6   description1  12149 non-null  object
dtypes: object(7)
memory usage: 672.7+ KB


lambda - 


group by product, order by time, and then see what would look like (poss window function)

### Analysis 

In [ ]:
#splitting out Day month year from reviewtime
clean_review_df['reviewTime'] = review_df['reviewTime'].str.replace(",","")
clean_review_df[['Day','Month',"Year"]] = clean_review_df["reviewTime"].str.split(" ", expand = True)
clean_review_df

In [ ]:
#separating columns of clean_review_df
clean_review_df[['Day','Month',"Year"]] = clean_review_df["reviewTime"].str.split(" ", expand = True)
clean_review_df

In [ ]:
#Looking at number of reviews over the years 
plt.figure(figsize=(10,8))
clean_review_df['Year'].value_counts().sort_index().plot()
plt.show()

In [ ]:
clean_review_df.groupby("Year")["Year"].value_counts()

In [ ]:
#unique product values 
unique_asin = clean_review_df["asin"].nunique()
print(unique_asin)

There are 12,120 unique asins in the data set. 

value_counts, look at percentile, 90% of the products have x number of reviews 

In [ ]:
clean_review_df.info()

In [ ]:
clean_review_df["asin"].value_counts()

In [ ]:
#histogram of reviews 
binwidth = 50

plt.figure(figsize=(10,8))
asin_data = clean_review_df["asin"].value_counts()
plt.hist(asin_data , bins=np.arange(0,3500, binwidth))

plt.xlabel(f'Review count. Bin Width: {binwidth}')
plt.ylabel('Frequency')
plt.title('Distribution of Review counts in dataset')
plt.show()

- consider removing products with value counts less than a certain amount. 

In [ ]:
clean_review_df.info()

In [ ]:
plot = clean_review_df.groupby("overall").count().reset_index()
plot = plot.rename(columns= {"overall":"Review Rating","verified":"Counts of Reviews"})

#plot structure
fig = px.bar(plot, 
             x = "Review Rating",
             y = "Counts of Reviews",
             title = "Amazon users are generous when they review, 65% of the dataset gave out a 5 star review",
             color = "Review Rating",
             color_continuous_scale="darkmint"
             
             )

fig.update_layout(coloraxis_showscale=False)

#plot 
fig.show()

In [1]:
357973/clean_review_df.shape[0]

NameError: name 'clean_review_df' is not defined

In [290]:
clean_review_df.shape

(539120, 11)

In [ ]:
#calculating % of duplicates
clean_review_df.duplicated().sum() / clean_review_df.shape[0] *100

{'category': [], 'tech1': '', 'description': ['After a long day of handling thorny situations, our new hand therapy pump is just the help you need. It contains shea butter as well as extracts of yarrow, clover and calendula to help soothe and condition work-roughened hands.', 'By Crabtree & Evelyn', 'The aromatic benefits of herbs are varied and far-reaching, so we combined a whole bunch of them into one restoratively fragrant line-up straight from the garden.', 'We&#039;ve formulated our Gardeners Hand Therapy with Myrrh Extract to help condition nails and cuticles as well as skin super hydrators macadamia seed oil and shea butter to help replenish lost moisture. Rich in herbal extracts like cooling cucumber and rosemary leaf  a favourite for antioxidants  to help protect hands against daily urban and environmental stresses while the hydrating power of Vitamin E, Hyaluronic Acid and Ceramides contribute to improve the skins natural moisture barrier with this garden-inspired treatment.

0
12299


(12299, 19)